<a href="https://colab.research.google.com/github/ofoofoo/dynamic-token-merging/blob/main/experiments/weighted_layerwise_merge_sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install ptflops
!pip install fvcore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rou

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import evaluate
from tqdm import tqdm
from ptflops import get_model_complexity_info

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import random
import time
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import evaluate
import torch
from fvcore.nn import FlopCountAnalysis

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn").to(device)

ds = load_dataset("abisee/cnn_dailymail", "3.0.0")
test_dataset = ds['test']
subset_size = 100
random_indices = random.sample(range(len(test_dataset)), subset_size)
test_dataset = test_dataset.select(random_indices)
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
total_flops = 0
total_time = 0
outputs = []
targets = []

In [ ]:
print("Vocab Size", model.config.vocab_size)
print("Hidden Size", model.config.hidden_size)
print(model.lm_head)

Vocab Size 50264
Hidden Size 1024
Linear(in_features=1024, out_features=50264, bias=False)


In [ ]:
'''
        num_pruning_steps = 3
        for i in range(num_pruning_steps):
            for j in range(12 // num_pruning_steps):
                result = model.model.encoder.layers[i * (12 // num_pruning_steps) + j](encoder_hidden_states,
                                                               attention_mask = None,
                                                               layer_head_mask=None,
                                                               output_attentions=True)
                encoder_hidden_states = result[0]
                attention = result[1]
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, 0)
            mask = avg_attention > threshold
            encoder_hidden_states = encoder_hidden_states[mask].unsqueeze(dim=0)
        '''

In [ ]:
for example in tqdm(test_dataset):
    article = example['article']
    reference_summary = example['highlights']
    tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids_tensor = torch.tensor(token_ids).unsqueeze(0).to(device)

    with torch.no_grad():
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        model.eval()
        start_time = time.time()
        print('')

        # Embedding as done in the Encoder in
        # https://huggingface.co/transformers/v3.0.2/_modules/transformers/modeling_bart.html#BartModel.forward
        inputs_embeds = model.model.encoder.embed_tokens(token_ids_tensor)
        embed_pos = model.model.encoder.embed_positions(token_ids_tensor)
        encoder_hidden_states = inputs_embeds + embed_pos
        encoder_hidden_states = model.model.encoder.layernorm_embedding(encoder_hidden_states)

        '''
        for i in range(12):
            layer_outputs = model.model.encoder.layers[i](encoder_hidden_states,
                                                   attention_mask = None,
                                                   layer_head_mask=None,
                                                   output_attentions=True)
            encoder_hidden_states = layer_outputs[0]
            attention = layer_outputs[1]
        '''

        num_pruning_steps = 3
        pruning_proportion = 0.2
        for i in range(num_pruning_steps):
            for j in range(12 // num_pruning_steps):
                current_layer_index = i * (12 // num_pruning_steps) + j
                layer_outputs = model.model.encoder.layers[current_layer_index](encoder_hidden_states,
                                                   attention_mask = None,
                                                   layer_head_mask=None,
                                                   output_attentions=True)
                encoder_hidden_states, attention = layer_outputs
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, pruning_proportion)
            mask = avg_attention > threshold
            encoder_hidden_states = encoder_hidden_states[mask].unsqueeze(dim=0)

        # encoder_outputs = model.model.encoder(input_ids = token_ids_tensor, output_attentions=True, output_hidden_states=True)
        # print("Encoder Outputs", encoder_outputs.last_hidden_state.shape)
        # print("Encoder Layer Outputs", encoder_hidden_states.shape)

        decoder_input_ids = torch.tensor([[0, 0]]).to(device)
        for _ in range(128):
            decoder_outputs = model.model.decoder(
                input_ids = decoder_input_ids,
                encoder_hidden_states = encoder_hidden_states
            )
            logits = model.lm_head(decoder_outputs[0])
            predicted_token_id = logits.argmax(dim=-1)[:, -1]
            decoder_input_ids = torch.cat([decoder_input_ids, predicted_token_id.unsqueeze(dim=0)], dim=-1)
            if predicted_token_id == 2:
                break

        generated_summary = tokenizer.decode(decoder_input_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        end_time = time.time()

        real_generated_tokens = model.generate(
            input_ids=token_ids_tensor,
            attention_mask=None
        )
        real_generated_summary = tokenizer.decode(real_generated_tokens[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

        print('')
        print(generated_summary)
        print('----------------')
        print(real_generated_summary)
        print('----------------')
        print(reference_summary)
        total_time += (end_time - start_time)
        outputs.append(generated_summary)
        targets.append(reference_summary)

average_time = total_time / len(test_dataset)

rouge_results = rouge.compute(predictions=outputs, references=targets)
bleu_results = bleu.compute(predictions=outputs, references=targets)

print("Inference Results:")
print(f"Average forward pass time: {average_time:.4f} seconds")
print("ROUGE Scores:", rouge_results)
print("BLEU Score:", bleu_results)

  0%|          | 0/100 [00:00<?, ?it/s]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



  1%|          | 1/100 [00:06<11:07,  6.74s/it]



----------------
Alleged rape took place on Panama City Beach, Florida, during Spring Break. Hundreds of people watched but did not intervene, authorities claim. Footage was uncovered on a cell phone during an unconnected investigation into a shooting in Troy, Alabama. Delone' Martistee, 22, and Ryan Austin Calhoun, 23, have been suspended from college while they are detained for questioning.
----------------
Police in Troy, Alabama, found the video while investigating a shooting .
Video 'shows two men raping unconscious woman on Florida beach'
Authorities say hundreds of people walk past but don't intervene .
Troy University students Delone' Martistee, 22, and Ryan Austin Calhoun, 23, have been arrested in connection with the alleged incident .
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



  2%|▏         | 2/100 [00:10<08:22,  5.13s/it]

...

 99%|█████████▉| 99/100 [04:59<00:03,  3.04s/it]


Aboriginal teenager who was arrested after anti-terror raids in Melbourne has complained he was victim of police brutality. Cruse, 19, one of five men taken into custody after raids on Saturday morning, said officers assaulted him after handcuffed. His father, Glen, said he was brutally attacked by police during the raids. It comes after the father of one of the arrested teens, said police held a gun and held a gun on his family.
----------------
Eathan Cruse, 19, was one of five men taken into custody after anti-terror raids in Melbourne on Saturday morning. He said officers assaulted him after he had been handcuffed. His father, Glen, also said he was brutally attacked by police during the raid, and his son was hospitalised as a result of injuries he sustained. Victoria Police confirmed a complaint had been made in regards to the treatment of Mr Cruse. The raids were carried out across Melbourne, and reportedly foiled an ANZAC Day terror plot.
----------------
Aboriginal teenager ar

100%|██████████| 100/100 [05:02<00:00,  3.02s/it]


Indian doctors at Kathmandu Hospital say Pakistan sent aid to Nepal. The aid included packets of ready-to-eat meals and food from a hotel. The majority of Nepal's population treats cows as sacred. Eating beef is a religious offence in Nepal.
----------------
Indian doctors at Kathmandu’s Bir Hospital told Mail Today that packets of ‘beef masala’ were sent by Pakistan on Tuesday as part of relief aid to the temblor survivors. The majority-Hindu country treats cows as sacred and there is a blanket ban on slaughtering the animal. The development has the potential to trigger diplomatic acrimony between the South Asian Association for Regional Cooperation member countries.
----------------
Cows are treated as sacred in Nepal, where the majority are Hindus .
Killing a cow used to be punishable by death, but now carries a 12-year jail term .
Nepalese government officials have informed the country's PM .
An internal inquiry is underway and the matter will be raised with Pakistan .


Inference Results:
Average forward pass time: 1.6576 seconds
ROUGE Scores: {'rouge1': 0.33024858946095503, 'rouge2': 0.13350941561362567, 'rougeL': 0.22530669726234168, 'rougeLsum': 0.2786344300013116}
BLEU Score: {'bleu': 0.09898970818268119, 'precisions': [0.4242881794650561, 0.1667033208709039, 0.08411843876177658, 0.044632639047837035], 'brevity_penalty': 0.7754482235457536, 'length_ratio': 0.7972484952708513, 'translation_length': 4636, 'reference_length': 5815}


In [ ]:
import torch.nn as nn
from fvcore.nn import FlopCountAnalysis
from transformers.modeling_outputs import BaseModelOutput

class PrunedEncoder(nn.Module):
    def __init__(self, encoder, num_pruning_steps=3, pruning_proportion=0):
        super(PrunedEncoder, self).__init__()
        self.encoder = encoder
        self.num_pruning_steps = num_pruning_steps
        self.pruning_proportion = pruning_proportion

    def forward(self, input_ids):
        inputs_embeds = self.encoder.embed_tokens(input_ids)
        embed_pos = self.encoder.embed_positions(input_ids)
        encoder_hidden_states = inputs_embeds + embed_pos
        encoder_hidden_states = self.encoder.layernorm_embedding(encoder_hidden_states)

        attention = None
        for i in range(self.num_pruning_steps):
            for j in range(12 // self.num_pruning_steps):
                current_layer_index = i * (12 // self.num_pruning_steps) + j
                layer_outputs = self.encoder.layers[current_layer_index](
                    encoder_hidden_states,
                    attention_mask=None,
                    layer_head_mask=None,
                    output_attentions=True
                )
                encoder_hidden_states, attention = layer_outputs[:2]
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, self.pruning_proportion)
            mask = avg_attention > threshold
            mask = mask.squeeze(0)
            encoder_hidden_states = encoder_hidden_states[:, mask, :]

        return encoder_hidden_states



class PrunedModel(nn.Module):
    def __init__(self, model, num_pruning_steps=3, pruning_proportion=0):
        super(PrunedModel, self).__init__()
        self.encoder = PrunedEncoder(model.model.encoder, num_pruning_steps, pruning_proportion)
        self.decoder = model.model.decoder
        self.lm_head = model.lm_head
        self.config = model.config

    def forward(self, input_ids, decoder_input_ids):
        encoder_outputs = self.encoder(input_ids)
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            encoder_hidden_states=encoder_outputs,
            encoder_attention_mask=None,
            past_key_values=None,
            use_cache=False,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
        sequence_output = decoder_outputs.last_hidden_state
        logits = self.lm_head(sequence_output)
        return logits


class ReferenceModel(nn.Module):
    def __init__(self, model):
        super(ReferenceModel, self).__init__()
        self.model = model

    def forward(self, input_ids, decoder_input_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=None,
            decoder_input_ids=decoder_input_ids,
            use_cache=False,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
        return outputs.logits

all_results = {}

for i in [0.2]:
  print(f"RUNNING WITH PRUNING PROPORTION: {i}")
  total_flops_pruned = 0
  total_flops_ref = 0
  total_time = 0
  total_time_ref = 0
  outputs = []
  outputs_ref = []
  targets = []

  pruned_model_instance = PrunedModel(model, pruning_proportion=i).to(device)
  reference_model_instance = ReferenceModel(model).to(device)

  for example in tqdm(test_dataset):
      article = example['article']
      reference_summary = example['highlights']
      tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
      token_ids = tokenizer.convert_tokens_to_ids(tokens)
      token_ids_tensor = torch.tensor([token_ids]).to(device)  # [1, seq_len]

      decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device)

      with torch.no_grad():
          print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
          model.eval()
          print('')

          # measure FLOPs: need to run this before timing
          flops_pruned = FlopCountAnalysis(
              pruned_model_instance,
              (token_ids_tensor, decoder_input_ids)
          ).total()
          total_flops_pruned += flops_pruned

          # pruned model generation
          start_time = time.time()
          generated_tokens = model.generate(
              encoder_outputs=BaseModelOutput(
                  last_hidden_state=pruned_model_instance.encoder(token_ids_tensor)
              ),
              decoder_input_ids=decoder_input_ids,
              max_length=128,
              num_beams=4,
              early_stopping=True
          )
          generated_summary = tokenizer.decode(
              generated_tokens[0],
              skip_special_tokens=True,
              clean_up_tokenization_spaces=True
          )
          end_time = time.time()
          total_time += (end_time - start_time)

          flops_ref = FlopCountAnalysis(
              reference_model_instance,
              (token_ids_tensor, decoder_input_ids)
          ).total()
          total_flops_ref += flops_ref

          # reference model generation
          start_time_ref = time.time()
          real_generated_tokens = model.generate(
              input_ids=token_ids_tensor,
              attention_mask=None,
              max_length=128,
              num_beams=4,
              early_stopping=True
          )
          real_generated_summary = tokenizer.decode(
              real_generated_tokens[0],
              skip_special_tokens=True,
              clean_up_tokenization_spaces=True
          )
          end_time_ref = time.time()
          total_time_ref += (end_time_ref - start_time_ref)

          print('')
          print(generated_summary)
          print('----------------')
          print(real_generated_summary)
          print('----------------')
          print(reference_summary)
          outputs.append(generated_summary)
          outputs_ref.append(real_generated_summary)
          targets.append(reference_summary)

  average_flops_pruned = total_flops_pruned / len(test_dataset)
  average_flops_ref = total_flops_ref / len(test_dataset)
  average_time = total_time / len(test_dataset)
  average_time_ref = total_time_ref / len(test_dataset)

  rouge_results = rouge.compute(predictions=outputs, references=targets)
  bleu_results = bleu.compute(predictions=outputs, references=targets)

  rouge_results_ref = rouge.compute(predictions=outputs_ref, references=targets)
  bleu_results_ref = bleu.compute(predictions=outputs_ref, references=targets)

  print("Inference Results:")
  print(f"Average forward pass time: {average_time:.4f} seconds")
  print(f"Average forward pass time (reference): {average_time_ref:.4f} seconds")
  print(f"Average FLOPs for pruned model: {average_flops_pruned:.2f}")
  print(f"Average FLOPs for reference model: {average_flops_ref:.2f}")
  print("ROUGE Scores:", rouge_results)
  print("ROUGE Scores (reference):", rouge_results_ref)
  print("BLEU Score:", bleu_results)
  print("BLEU Score (reference):", bleu_results_ref)

  summary_stats = {"average_flops_pruned": average_flops_pruned,
      "average_flops_ref": average_flops_ref,
      "average_time_pruned": average_time,
      "average_time_ref": average_time_ref,
      "average_rouge_pruned": rouge_results,
      "average_rouge_ref": rouge_results_ref,
      "average_bleu_pruned": bleu_results,
      "average_bleu_ref": bleu_results_ref}
  all_results[i] = summary_stats

RUNNING WITH PRUNING PROPORTION: 0.2


  0%|          | 0/100 [00:00<?, ?it/s]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



BartModel is using BartSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `layer_head_mask` not None. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
model.model.shared
  1%|          | 1/100 [00:05<08:41,  5.27s/it]


'My my! At Napoleon did surrender,’ sang Sweden’s fab four in their famous Eurovision Song Contest winner. But Napoleon certainly didn’t surrender at Waterloo. The British and the British remain fascinated by the victory. But while the British know happened when Napoleon met his Waterloo, the French-speaking world is less convinced.
----------------
'My, my! At Waterloo Napoleon did surrender,’ sang Sweden’s fab four in their famous Eurovision Song Contest winner. But Napoleon certainly didn’t surrender at Waterloo. The British – and the English-speaking nations of the world – remain fascinated by the victory and the bloody manner in which it was achieved.
----------------
Contrary to the Abba  song, Napoleon did not surrender at Waterloo .
Templar hospice is being turned into a restaurant-cum-brewery, museum .
May sees the 75th anniversary of 'glorious failure' of Operation Dynamo .


KeyboardInterrupt: 

In [ ]:
from sys import set_coroutine_origin_tracking_depth
# DEREK

def weighted_merge(relative_weight, merge_to_closest):
    def return_function(encoder_hidden_states, anchors):
        # relative_weight (range from 0 to 1) is the relative weight of the non-salient tokens to the anchors
        # if relative_weight == 1, then this corresponds to normal averaging
        # if relative_weight == 0, then non-salient tokens are pruned out

        # merge_to_closest (boolean) is True if non-anchors get merged into their closest anchors
        #      and False if they get merged into the closest right anchor
        seq_len = encoder_hidden_states.size(1)
        merged_hidden_states = []
        reverse_relative_weight = 1/relative_weight - 1 if not relative_weight == 0 else None
        for ai, anchor in enumerate(anchors):
            left_bound, right_bound = None, None
            if ai == 0: left_bound = 0
            else:
                if merge_to_closest: left_bound = (anchor + anchors[ai - 1] + 1) // 2
                else: left_bound = anchors[ai - 1] + 1
            if ai == len(anchors) - 1: right_bound = seq_len
            else:
                if merge_to_closest: right_bound = (anchor + anchors[ai + 1] + 1) // 2
                else: right_bound = anchor + 1

            weighted_average = None
            if relative_weight == 0:
                weighted_average = encoder_hidden_states[:, anchor:anchor + 1, :]
            else:
                merging_section = encoder_hidden_states[:, left_bound:right_bound, :]
                anchor_section = encoder_hidden_states[:, anchor:anchor + 1, :]
                section_sum = merging_section.sum(dim=1, keepdim=True) + anchor_section * reverse_relative_weight
                weighted_average = section_sum / (right_bound - left_bound + reverse_relative_weight)
            merged_hidden_states.append(weighted_average)
            '''
            anchor_section = encoder_hidden_states[:, anchor:anchor + 1, :]
            section_length = right_bound - left_bound
            weighted_average = (torch.sum(merging_section, dim=1) * relative_weight + torch.sum(anchor_section, dim=1) * (1 - relative_weight)) / ((section_length - 1) * relative_weight + 1)
            #weights = torch.ones_like(merging_section) * relative_weight
            #weights[:, anchor - left_bound, :] = 1
            #weighted_average = torch.sum(merging_section * weights, dim=1)/torch.sum(weights, dim=1)
            merged_hidden_states.append(weighted_average.unsqueeze(dim=0))
            '''
        return torch.cat(merged_hidden_states, dim=1)
    return return_function

In [ ]:
def temp_scaled_merge(temp, merge_to_closest):
    def return_function(encoder_hidden_states, anchors):
        seq_len = encoder_hidden_states.size(1)
        merged_hidden_states = []
        for idx, anchor in enumerate(anchors):
            left_bound, right_bound = None, None
            if idx == 0: left_bound = 0
            else:
                if merge_to_closest: left_bound = (anchor + anchors[idx - 1] + 1) // 2
                else: left_bound = anchors[idx - 1] + 1
            if idx == len(anchors) - 1: right_bound = seq_len
            else:
                if merge_to_closest: right_bound = (anchor + anchors[idx + 1] + 1) // 2
                else: right_bound = anchor + 1
            
            merging_section = encoder_hidden_states[:, left_bound:right_bound, :] #(batch_size, merging_section, hidden_dim)
            logits = torch.sum(merging_section, dim = 2, keepdim = True) #(batch_size, merging_section, 1)

            temp_scaled_weights = torch.nn.functional.softmax(merging_section/temp, dim = 1) #(batch_size, merging_section, 1)

            temp_scaled_average = torch.sum(merging_section * temp_scaled_weights, dim=1, keepdim=True) #(batch_size, 1, hidden_dim)
            merged_hidden_states.append(temp_scaled_average)
        return torch.cat(merged_hidden_states, dim=1) #(batch_size, anchors, hidden_dim)
    return return_function

In [ ]:
# MERGING INSTEAD OF PRUNING

import torch
import torch.nn as nn
from fvcore.nn import FlopCountAnalysis
from transformers.modeling_outputs import BaseModelOutput
from tqdm import tqdm

class PrunedEncoder(nn.Module):
    def __init__(self, encoder, num_pruning_steps=3, pruning_proportion=0, merge_function=None):
        super(PrunedEncoder, self).__init__()
        self.encoder = encoder
        self.num_pruning_steps = num_pruning_steps
        self.pruning_proportion = pruning_proportion
        if merge_function == None: self.merge_function = weighted_merge(1, False)
        else: self.merge_function = merge_function

    def forward(self, input_ids):
        inputs_embeds = self.encoder.embed_tokens(input_ids)
        embed_pos = self.encoder.embed_positions(input_ids)
        encoder_hidden_states = inputs_embeds + embed_pos
        encoder_hidden_states = self.encoder.layernorm_embedding(encoder_hidden_states)

        attention = None
        for i in range(self.num_pruning_steps):
            for j in range(12 // self.num_pruning_steps):
                current_layer_index = i * (12 // self.num_pruning_steps) + j
                layer_outputs = self.encoder.layers[current_layer_index](
                    encoder_hidden_states,
                    attention_mask=None,
                    layer_head_mask=None,
                    output_attentions=True
                )
                encoder_hidden_states, attention = layer_outputs[:2]
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, self.pruning_proportion)
            mask = avg_attention > threshold
            mask = mask.squeeze(0) # [seq_len]
            #encoder_hidden_states = encoder_hidden_states[:, mask, :]
            anchors = torch.where(mask)[0]  # positions of significant tokens
            new_hidden_states = []
            current_pos = 0
            seq_len = encoder_hidden_states.size(1)
            #self.merge_function(encoder_hidden_states, anchors)
            encoder_hidden_states = self.merge_function(encoder_hidden_states, anchors)

            '''
            for anchor_pos in anchors:
                if current_pos < anchor_pos:
                    avg_token = encoder_hidden_states[:, current_pos:anchor_pos + 1, :].mean(dim=1, keepdim=True) # current_pos:anchor_pos + 1 MERGES ALL INSIGNIFICANT TOKENS TO THE ANCHOR TOKEN
                    new_hidden_states.append(avg_token)
                # for appending the anchor token
                anchor_token = encoder_hidden_states[:, anchor_pos:anchor_pos+1, :]
                new_hidden_states.append(anchor_token)
                current_pos = anchor_pos + 1

            if current_pos < seq_len:
                avg_token = encoder_hidden_states[:, current_pos:, :].mean(dim=1, keepdim=True)
                new_hidden_states.append(avg_token)
            encoder_hidden_states = torch.cat(new_hidden_states, dim=1)
            '''

        return encoder_hidden_states


class PrunedModel(nn.Module):
    def __init__(self, model, num_pruning_steps=3, pruning_proportion=0, merge_function=None):
        super(PrunedModel, self).__init__()
        self.encoder = PrunedEncoder(model.model.encoder, num_pruning_steps, pruning_proportion, merge_function)
        self.decoder = model.model.decoder
        self.lm_head = model.lm_head
        self.config = model.config

    def forward(self, input_ids, decoder_input_ids):
        encoder_outputs = self.encoder(input_ids)
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            encoder_hidden_states=encoder_outputs,
            encoder_attention_mask=None,
            past_key_values=None,
            use_cache=False,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
        sequence_output = decoder_outputs.last_hidden_state
        logits = self.lm_head(sequence_output)
        return logits


class ReferenceModel(nn.Module):
    def __init__(self, model):
        super(ReferenceModel, self).__init__()
        self.model = model

    def forward(self, input_ids, decoder_input_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=None,
            decoder_input_ids=decoder_input_ids,
            use_cache=False,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
        return outputs.logits

In [ ]:
all_results = {}

for w in [0.2, 0.4, 0.6, 0.8, 1]:
    print(f"RUNNING WITH NON-ANCHOR WEIGHTING: {w}")
    total_flops_pruned = 0
    total_flops_ref = 0
    total_time = 0
    total_time_ref = 0
    outputs = []
    outputs_ref = []
    targets = []

    merge_function = weighted_merge(w, False)
    pruned_model_instance = PrunedModel(model, pruning_proportion=0.9, merge_function=merge_function).to(device)
    reference_model_instance = ReferenceModel(model).to(device)
    for example in tqdm(test_dataset):
        article = example['article']
        reference_summary = example['highlights']
        tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        token_ids_tensor = torch.tensor([token_ids]).to(device)  # [1, seq_len]

        decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device)

        with torch.no_grad():
            print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
            model.eval()
            print('')

            # measure FLOPs: need to run this before timing
            flops_pruned = FlopCountAnalysis(
                pruned_model_instance,
                (token_ids_tensor, decoder_input_ids)
            ).total()
            total_flops_pruned += flops_pruned

            # pruned model generation
            model.eval()
            start_time = time.time()
            generated_tokens = model.generate(
                encoder_outputs=BaseModelOutput(
                    last_hidden_state=pruned_model_instance.encoder(token_ids_tensor)
                ),
                decoder_input_ids=decoder_input_ids,
                max_length=128,
                num_beams=4,
                early_stopping=True
            )
            generated_summary = tokenizer.decode(
                generated_tokens[0],
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )
            end_time = time.time()
            total_time += (end_time - start_time)

            flops_ref = FlopCountAnalysis(
                reference_model_instance,
                (token_ids_tensor, decoder_input_ids)
            ).total()
            total_flops_ref += flops_ref

            # reference model generation
            start_time_ref = time.time()
            real_generated_tokens = model.generate(
                input_ids=token_ids_tensor,
                attention_mask=None,
                max_length=128,
                num_beams=4,
                early_stopping=True
            )
            real_generated_summary = tokenizer.decode(
                real_generated_tokens[0],
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )
            end_time_ref = time.time()
            total_time_ref += (end_time_ref - start_time_ref)

            print('')
            print(generated_summary)
            print('----------------')
            print(real_generated_summary)
            print('----------------')
            print(reference_summary)
            print(end_time - start_time, end_time_ref - start_time_ref)
            outputs.append(generated_summary)
            outputs_ref.append(real_generated_summary)
            targets.append(reference_summary)
    average_flops_pruned = total_flops_pruned / len(test_dataset)
    average_flops_ref = total_flops_ref / len(test_dataset)
    average_time = total_time / len(test_dataset)
    average_time_ref = total_time_ref / len(test_dataset)

    rouge_results = rouge.compute(predictions=outputs, references=targets)
    bleu_results = bleu.compute(predictions=outputs, references=targets)

    rouge_results_ref = rouge.compute(predictions=outputs_ref, references=targets)
    bleu_results_ref = bleu.compute(predictions=outputs_ref, references=targets)

    print("Inference Results:")
    print(f"Average forward pass time: {average_time:.4f} seconds")
    print(f"Average forward pass time (reference): {average_time_ref:.4f} seconds")
    print(f"Average FLOPs for pruned model: {average_flops_pruned:.2f}")
    print(f"Average FLOPs for reference model: {average_flops_ref:.2f}")
    print("ROUGE Scores:", rouge_results)
    print("ROUGE Scores (reference):", rouge_results_ref)
    print("BLEU Score:", bleu_results)
    print("BLEU Score (reference):", bleu_results_ref)

    summary_stats = {"average_flops_pruned": average_flops_pruned,
        "average_flops_ref": average_flops_ref,
        "average_time_pruned": average_time,
        "average_time_ref": average_time_ref,
        "average_rouge_pruned": rouge_results,
        "average_rouge_ref": rouge_results_ref,
        "average_bleu_pruned": bleu_results,
        "average_bleu_ref": bleu_results_ref}
    all_results[w] = summary_stats

RUNNING WITH NON-ANCHOR WEIGHTING: 0.2


  0%|          | 0/100 [00:00<?, ?it/s]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



model.model.shared
  1%|          | 1/100 [00:05<08:56,  5.42s/it]


The first of a two-day holiday weekend is set to be marked by the release of a new film, The Descendants of the Holy Grail. The film is based on the novel by the same name. The movie is about the life of a young boy who is raised by his father after his father is killed in a car crash.
----------------
Ollie Devoto and Semesa Rokoduguni scored tries for Bath. Sam Burgess made his first Premiership start at number six. Newcastle responded with a Sinoti Sinoti try. Anthony Watson's score on the stroke of half-time earned Bath a 19-13 lead at the interval. Matt Banahan went over in the 42nd minute to give the visitors their bonus point.
----------------
Ollie Devoto, Semesa Rokoduguni, Anthony Watson and Matt Banahan all crossed for Bath .
Bath fly half George Ford added nine points from the boot .
Newcastle replied through a Sinoti Sinoti try .
1.3909988403320312 1.5774903297424316
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



...


The first half of this year was the busiest in the history of the U.S. Olympic Games. The second half of the year was one of the slowest in Olympic history. The first half was dominated by the United States, but the second half was much more competitive.
----------------
Ryan Bertrand has made 26 Premier League appearances for Southampton so far this season. The left back has also forced his way back into the England squad. Bertrand took to Instagram to upload a snap of himself and his daughter. The 25-year-old got his little girl a red Ferrari just like his own.
----------------
Ryan Bertrand takes to Instagram to upload snap of his daughters toy car .
25-year-old Southampton defender got her a red Ferrari to match his own .
Bertrand has made 26 Premier League appearances this season .
1.0176606178283691 1.209669828414917
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



model.model.shared
  3%|▎         | 3/100 [00:15<08:09,  5.05s/it]


The first half of the year has been a difficult one for the U.S. economy. The second half has been the most successful in the world. The U.K. is the only country in Europe to have had a better year than the United States in the first half.
----------------
Aaron Ramsey fired Arsenal into a 1-0 lead in the 12th minute at Turf Moor. The Welsh midfielder scored his ninth goal of the season to put the Gunners in the lead. Kieran Trippier pulled one back for Burnley with a 25-yard free-kick shortly after Ramsey's goal. Arsene Wenger's side have now won eight games in a row for the first time since The Invincibles in 2004.
----------------
Arsenal earn 1-0 Premier League victory against Burnley at Turf Moor .
Aaron Ramsey gives Gunners an early lead with 12th minute strike .
Gunners close to gap to four points behind Premier League leaders Chelsea .
Burnley remain in the bottom three with fixtures against fellow strugglers Leicester, Hull and Aston Villa to come .
CLICK HERE for full player

model.model.shared
  3%|▎         | 3/100 [00:18<10:03,  6.22s/it]


KeyboardInterrupt: 

In [ ]:
all_results = {}

for temp in [0.2, 0.4, 0.6, 0.8, 1]:
    print(f"RUNNING TEMPERATURE-SCALED LOGITS WITH TEMPERATURE: {temp}")
    total_flops_pruned = 0
    total_flops_ref = 0
    total_time = 0
    total_time_ref = 0
    outputs = []
    outputs_ref = []
    targets = []

    merge_function = temp_scaled_merge(temp, False)
    pruned_model_instance = PrunedModel(model, pruning_proportion=0.4, merge_function=merge_function).to(device)
    reference_model_instance = ReferenceModel(model).to(device)

    for example in tqdm(test_dataset):
        article = example['article']
        reference_summary = example['highlights']
        tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        token_ids_tensor = torch.tensor([token_ids]).to(device)  # [1, seq_len]

        decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device)

        with torch.no_grad():
            print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
            model.eval()
            print('')

            # measure FLOPs: need to run this before timing
            flops_pruned = FlopCountAnalysis(
                pruned_model_instance,
                (token_ids_tensor, decoder_input_ids)
            ).total()
            total_flops_pruned += flops_pruned

            # pruned model generation
            model.eval()
            start_time = time.time()
            generated_tokens = model.generate(
                encoder_outputs=BaseModelOutput(
                    last_hidden_state=pruned_model_instance.encoder(token_ids_tensor)
                ),
                decoder_input_ids=decoder_input_ids,
                max_length=128,
                num_beams=4,
                early_stopping=True
            )
            generated_summary = tokenizer.decode(
                generated_tokens[0],
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )
            end_time = time.time()
            total_time += (end_time - start_time)

            flops_ref = FlopCountAnalysis(
                reference_model_instance,
                (token_ids_tensor, decoder_input_ids)
            ).total()
            total_flops_ref += flops_ref

            # reference model generation
            start_time_ref = time.time()
            real_generated_tokens = model.generate(
                input_ids=token_ids_tensor,
                attention_mask=None,
                max_length=128,
                num_beams=4,
                early_stopping=True
            )
            real_generated_summary = tokenizer.decode(
                real_generated_tokens[0],
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )
            end_time_ref = time.time()
            total_time_ref += (end_time_ref - start_time_ref)

            print('')
            print(generated_summary)
            print('----------------')
            print(real_generated_summary)
            print('----------------')
            print(reference_summary)
            print(end_time - start_time, end_time_ref - start_time_ref)
            outputs.append(generated_summary)
            outputs_ref.append(real_generated_summary)
            targets.append(reference_summary)
    average_flops_pruned = total_flops_pruned / len(test_dataset)
    average_flops_ref = total_flops_ref / len(test_dataset)
    average_time = total_time / len(test_dataset)
    average_time_ref = total_time_ref / len(test_dataset)

    rouge_results = rouge.compute(predictions=outputs, references=targets)
    bleu_results = bleu.compute(predictions=outputs, references=targets)

    rouge_results_ref = rouge.compute(predictions=outputs_ref, references=targets)
    bleu_results_ref = bleu.compute(predictions=outputs_ref, references=targets)

    print("Inference Results:")
    print(f"Average forward pass time: {average_time:.4f} seconds")
    print(f"Average forward pass time (reference): {average_time_ref:.4f} seconds")
    print(f"Average FLOPs for pruned model: {average_flops_pruned:.2f}")
    print(f"Average FLOPs for reference model: {average_flops_ref:.2f}")
    print("ROUGE Scores:", rouge_results)
    print("ROUGE Scores (reference):", rouge_results_ref)
    print("BLEU Score:", bleu_results)
    print("BLEU Score (reference):", bleu_results_ref)

    summary_stats = {"average_flops_pruned": average_flops_pruned,
        "average_flops_ref": average_flops_ref,
        "average_time_pruned": average_time,
        "average_time_ref": average_time_ref,
        "average_rouge_pruned": rouge_results,
        "average_rouge_ref": rouge_results_ref,
        "average_bleu_pruned": bleu_results,
        "average_bleu_ref": bleu_results_ref}
    all_results[f"temp {temp}"] = summary_stats

In [ ]:
print(all_results)
save = {0: {'average_flops_pruned': 92177895628.8, 'average_flops_ref': 130173797212.16, 'average_time_pruned': 1.3366907548904419, 'average_time_ref': 1.6318064832687378, 'average_rouge_pruned': {'rouge1': 0.36630161809982353, 'rouge2': 0.1275956660999049, 'rougeL': 0.23478703305551202, 'rougeLsum': 0.30243706966780953}, 'average_rouge_ref': {'rouge1': 0.41788733325640703, 'rouge2': 0.2070445301914161, 'rougeL': 0.29181900250783, 'rougeLsum': 0.3568827547260355}, 'average_bleu_pruned': {'bleu': 0.09255050006599924, 'precisions': [0.37323592893906693, 0.12561202093533683, 0.05632835308260347, 0.027782631486982352], 'brevity_penalty': 1.0, 'length_ratio': 1.0688553682342503, 'translation_length': 6023, 'reference_length': 5635}, 'average_bleu_ref': {'bleu': 0.14468862775427296, 'precisions': [0.3753209893228815, 0.17399643786820113, 0.10223642172523961, 0.06564304831666432], 'brevity_penalty': 1.0, 'length_ratio': 1.3130434782608695, 'translation_length': 7399, 'reference_length': 5635}}, 0.2: {'average_flops_pruned': 92177895628.8, 'average_flops_ref': 130173797212.16, 'average_time_pruned': 1.5448491644859315, 'average_time_ref': 1.637245614528656, 'average_rouge_pruned': {'rouge1': 0.35888000280956645, 'rouge2': 0.1227939880036868, 'rougeL': 0.22857577129211906, 'rougeLsum': 0.2912041192920526}, 'average_rouge_ref': {'rouge1': 0.41788733325640703, 'rouge2': 0.2070445301914161, 'rougeL': 0.29181900250783, 'rougeLsum': 0.3568827547260355}, 'average_bleu_pruned': {'bleu': 0.08736652950330923, 'precisions': [0.35603174603174603, 0.11306451612903226, 0.052950819672131146, 0.027333333333333334], 'brevity_penalty': 1.0, 'length_ratio': 1.1180124223602483, 'translation_length': 6300, 'reference_length': 5635}, 'average_bleu_ref': {'bleu': 0.14468862775427296, 'precisions': [0.3753209893228815, 0.17399643786820113, 0.10223642172523961, 0.06564304831666432], 'brevity_penalty': 1.0, 'length_ratio': 1.3130434782608695, 'translation_length': 7399, 'reference_length': 5635}}, 0.4: {'average_flops_pruned': 92177895628.8, 'average_flops_ref': 130173797212.16, 'average_time_pruned': 1.525521490573883, 'average_time_ref': 1.6267850518226623, 'average_rouge_pruned': {'rouge1': 0.3613007782752609, 'rouge2': 0.1293808351651808, 'rougeL': 0.2375624919133607, 'rougeLsum': 0.2993299026848748}, 'average_rouge_ref': {'rouge1': 0.41788733325640703, 'rouge2': 0.2070445301914161, 'rougeL': 0.29181900250783, 'rougeLsum': 0.3568827547260355}, 'average_bleu_pruned': {'bleu': 0.09256501939741578, 'precisions': [0.3577759871071716, 0.12022932022932023, 0.05661948376353039, 0.03014394580863675], 'brevity_penalty': 1.0, 'length_ratio': 1.101153504880213, 'translation_length': 6205, 'reference_length': 5635}, 'average_bleu_ref': {'bleu': 0.14468862775427296, 'precisions': [0.3753209893228815, 0.17399643786820113, 0.10223642172523961, 0.06564304831666432], 'brevity_penalty': 1.0, 'length_ratio': 1.3130434782608695, 'translation_length': 7399, 'reference_length': 5635}}, 0.6: {'average_flops_pruned': 92177895628.8, 'average_flops_ref': 130173797212.16, 'average_time_pruned': 1.529856562614441, 'average_time_ref': 1.6403126573562623, 'average_rouge_pruned': {'rouge1': 0.3665311703612486, 'rouge2': 0.1361681470536938, 'rougeL': 0.24345729500545904, 'rougeLsum': 0.30594176366845327}, 'average_rouge_ref': {'rouge1': 0.41788733325640703, 'rouge2': 0.2070445301914161, 'rougeL': 0.29181900250783, 'rougeLsum': 0.3568827547260355}, 'average_bleu_pruned': {'bleu': 0.10079499152925085, 'precisions': [0.3667315175097276, 0.1287079762689519, 0.06384048257372654, 0.03425357873210634], 'brevity_penalty': 1.0, 'length_ratio': 1.0945874001774623, 'translation_length': 6168, 'reference_length': 5635}, 'average_bleu_ref': {'bleu': 0.14468862775427296, 'precisions': [0.3753209893228815, 0.17399643786820113, 0.10223642172523961, 0.06564304831666432], 'brevity_penalty': 1.0, 'length_ratio': 1.3130434782608695, 'translation_length': 7399, 'reference_length': 5635}}, 0.8: {'average_flops_pruned': 92177895628.8, 'average_flops_ref': 130173797212.16, 'average_time_pruned': 1.498060793876648, 'average_time_ref': 1.619826853275299, 'average_rouge_pruned': {'rouge1': 0.37065324656452064, 'rouge2': 0.13989051444280554, 'rougeL': 0.24333424891556477, 'rougeLsum': 0.3104257874349211}, 'average_rouge_ref': {'rouge1': 0.41788733325640703, 'rouge2': 0.2070445301914161, 'rougeL': 0.29181900250783, 'rougeLsum': 0.3568827547260355}, 'average_bleu_pruned': {'bleu': 0.10551562013450233, 'precisions': [0.3744449925999013, 0.13292091623474336, 0.066655330726067, 0.03736377789309808], 'brevity_penalty': 1.0, 'length_ratio': 1.079148181011535, 'translation_length': 6081, 'reference_length': 5635}, 'average_bleu_ref': {'bleu': 0.14468862775427296, 'precisions': [0.3753209893228815, 0.17399643786820113, 0.10223642172523961, 0.06564304831666432], 'brevity_penalty': 1.0, 'length_ratio': 1.3130434782608695, 'translation_length': 7399, 'reference_length': 5635}}, 1: {'average_flops_pruned': 92177895628.8, 'average_flops_ref': 130173797212.16, 'average_time_pruned': 1.5116565918922424, 'average_time_ref': 1.6149716877937317, 'average_rouge_pruned': {'rouge1': 0.3729978901897434, 'rouge2': 0.13949257503986043, 'rougeL': 0.24815683859117538, 'rougeLsum': 0.3100091227842934}, 'average_rouge_ref': {'rouge1': 0.41788733325640703, 'rouge2': 0.2070445301914161, 'rougeL': 0.29181900250783, 'rougeLsum': 0.3568827547260355}, 'average_bleu_pruned': {'bleu': 0.10279023172219652, 'precisions': [0.3770092547491476, 0.1323650767453375, 0.06393690216479275, 0.034988905956647894], 'brevity_penalty': 1.0, 'length_ratio': 1.0929902395740905, 'translation_length': 6159, 'reference_length': 5635}, 'average_bleu_ref': {'bleu': 0.14468862775427296, 'precisions': [0.3753209893228815, 0.17399643786820113, 0.10223642172523961, 0.06564304831666432], 'brevity_penalty': 1.0, 'length_ratio': 1.3130434782608695, 'translation_length': 7399, 'reference_length': 5635}}}
initial_temps = {'temp 0.2': {'average_flops_pruned': 83727006586.88, 'average_flops_ref': 129100198615.04, 'average_time_pruned': 1.2262133860588074, 'average_time_ref': 1.5154845523834228, 'average_rouge_pruned': {'rouge1': 0.37392314692433215, 'rouge2': 0.12051555078010587, 'rougeL': 0.24061810684219473, 'rougeLsum': 0.3041732311883909}, 'average_rouge_ref': {'rouge1': 0.41858149947701984, 'rouge2': 0.19592063246471425, 'rougeL': 0.2939798241481364, 'rougeLsum': 0.35365387150961747}, 'average_bleu_pruned': {'bleu': 0.09388592223902621, 'precisions': [0.3878663940013633, 0.12742718446601942, 0.05575158786167961, 0.02819683908045977], 'brevity_penalty': 1.0, 'length_ratio': 1.0400567174760724, 'translation_length': 5868, 'reference_length': 5642}, 'average_bleu_ref': {'bleu': 0.15434915398574175, 'precisions': [0.3787956767884714, 0.1750521376433785, 0.1089540412044374, 0.07855995717344753], 'brevity_penalty': 1.0, 'length_ratio': 1.3775257001063452, 'translation_length': 7772, 'reference_length': 5642}}, 'temp 0.4': {'average_flops_pruned': 83727006586.88, 'average_flops_ref': 129100198615.04, 'average_time_pruned': 1.2218461799621583, 'average_time_ref': 1.5168952012062074, 'average_rouge_pruned': {'rouge1': 0.36537423561666865, 'rouge2': 0.11909848007142852, 'rougeL': 0.23650881912255461, 'rougeLsum': 0.30037004043975524}, 'average_rouge_ref': {'rouge1': 0.41858149947701984, 'rouge2': 0.19592063246471425, 'rougeL': 0.2939798241481364, 'rougeLsum': 0.35365387150961747}, 'average_bleu_pruned': {'bleu': 0.08831756588682019, 'precisions': [0.3801906058543227, 0.12413434903047091, 0.052854122621564484, 0.024390243902439025], 'brevity_penalty': 1.0, 'length_ratio': 1.0414746543778801, 'translation_length': 5876, 'reference_length': 5642}, 'average_bleu_ref': {'bleu': 0.15434915398574175, 'precisions': [0.3787956767884714, 0.1750521376433785, 0.1089540412044374, 0.07855995717344753], 'brevity_penalty': 1.0, 'length_ratio': 1.3775257001063452, 'translation_length': 7772, 'reference_length': 5642}}, 'temp 0.6': {'average_flops_pruned': 83727006586.88, 'average_flops_ref': 129100198615.04, 'average_time_pruned': 1.2159048342704772, 'average_time_ref': 1.5081881141662599, 'average_rouge_pruned': {'rouge1': 0.37167320916083413, 'rouge2': 0.12246977828710576, 'rougeL': 0.23961688447338841, 'rougeLsum': 0.30688751849672813}, 'average_rouge_ref': {'rouge1': 0.41858149947701984, 'rouge2': 0.19592063246471425, 'rougeL': 0.2939798241481364, 'rougeLsum': 0.35365387150961747}, 'average_bleu_pruned': {'bleu': 0.09188686801554202, 'precisions': [0.3825136612021858, 0.12769284225156358, 0.055162659123055166, 0.026457883369330453], 'brevity_penalty': 1.0, 'length_ratio': 1.0379298121233604, 'translation_length': 5856, 'reference_length': 5642}, 'average_bleu_ref': {'bleu': 0.15434915398574175, 'precisions': [0.3787956767884714, 0.1750521376433785, 0.1089540412044374, 0.07855995717344753], 'brevity_penalty': 1.0, 'length_ratio': 1.3775257001063452, 'translation_length': 7772, 'reference_length': 5642}}, 'temp 0.8': {'average_flops_pruned': 83727006586.88, 'average_flops_ref': 129100198615.04, 'average_time_pruned': 1.244049301147461, 'average_time_ref': 1.5162114429473876, 'average_rouge_pruned': {'rouge1': 0.3666177511740999, 'rouge2': 0.12221451889384416, 'rougeL': 0.24071707950513577, 'rougeLsum': 0.3027578854983405}, 'average_rouge_ref': {'rouge1': 0.41858149947701984, 'rouge2': 0.19592063246471425, 'rougeL': 0.2939798241481364, 'rougeLsum': 0.35365387150961747}, 'average_bleu_pruned': {'bleu': 0.0897859833319244, 'precisions': [0.3762541806020067, 0.12363945578231293, 0.05397923875432526, 0.025880281690140845], 'brevity_penalty': 1.0, 'length_ratio': 1.0599078341013826, 'translation_length': 5980, 'reference_length': 5642}, 'average_bleu_ref': {'bleu': 0.15434915398574175, 'precisions': [0.3787956767884714, 0.1750521376433785, 0.1089540412044374, 0.07855995717344753], 'brevity_penalty': 1.0, 'length_ratio': 1.3775257001063452, 'translation_length': 7772, 'reference_length': 5642}}, 'temp 1': {'average_flops_pruned': 83727006586.88, 'average_flops_ref': 129100198615.04, 'average_time_pruned': 1.215048463344574, 'average_time_ref': 1.5024070429801941, 'average_rouge_pruned': {'rouge1': 0.35914025683064255, 'rouge2': 0.12097034523033381, 'rougeL': 0.2368170308294883, 'rougeLsum': 0.29898338118648704}, 'average_rouge_ref': {'rouge1': 0.41858149947701984, 'rouge2': 0.19592063246471425, 'rougeL': 0.2939798241481364, 'rougeLsum': 0.35365387150961747}, 'average_bleu_pruned': {'bleu': 0.09178706246846255, 'precisions': [0.37506439979392064, 0.12598287611392625, 0.055308554152587584, 0.027159152634437807], 'brevity_penalty': 1.0, 'length_ratio': 1.032080822403403, 'translation_length': 5823, 'reference_length': 5642}, 'average_bleu_ref': {'bleu': 0.15434915398574175, 'precisions': [0.3787956767884714, 0.1750521376433785, 0.1089540412044374, 0.07855995717344753], 'brevity_penalty': 1.0, 'length_ratio': 1.3775257001063452, 'translation_length': 7772, 'reference_length': 5642}}}

{0: {'average_flops_pruned': 92177895628.8, 'average_flops_ref': 130173797212.16, 'average_time_pruned': 1.3366907548904419, 'average_time_ref': 1.6318064832687378, 'average_rouge_pruned': {'rouge1': 0.36630161809982353, 'rouge2': 0.1275956660999049, 'rougeL': 0.23478703305551202, 'rougeLsum': 0.30243706966780953}, 'average_rouge_ref': {'rouge1': 0.41788733325640703, 'rouge2': 0.2070445301914161, 'rougeL': 0.29181900250783, 'rougeLsum': 0.3568827547260355}, 'average_bleu_pruned': {'bleu': 0.09255050006599924, 'precisions': [0.37323592893906693, 0.12561202093533683, 0.05632835308260347, 0.027782631486982352], 'brevity_penalty': 1.0, 'length_ratio': 1.0688553682342503, 'translation_length': 6023, 'reference_length': 5635}, 'average_bleu_ref': {'bleu': 0.14468862775427296, 'precisions': [0.3753209893228815, 0.17399643786820113, 0.10223642172523961, 0.06564304831666432], 'brevity_penalty': 1.0, 'length_ratio': 1.3130434782608695, 'translation_length': 7399, 'reference_length': 5635}}, 0.2:

In [ ]:
all_results = {}

for w, mc in [(0.25, False), (0.25, True), (0.5, False), (0.5, True)]:
    print(f"RUNNING WITH NON-ANCHOR WEIGHTING {w} AND MERGING-TO-CLOSEST {mc}")
    total_flops_pruned = 0
    total_flops_ref = 0
    total_time = 0
    total_time_ref = 0
    outputs = []
    outputs_ref = []
    targets = []

    merge_function = weighted_merge(w, mc)
    pruned_model_instance = PrunedModel(model, pruning_proportion=0.35, merge_function=merge_function).to(device)
    reference_model_instance = ReferenceModel(model).to(device)
    for example in tqdm(test_dataset):
        article = example['article']
        reference_summary = example['highlights']
        tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        token_ids_tensor = torch.tensor([token_ids]).to(device)  # [1, seq_len]

        decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device)

        with torch.no_grad():
            print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
            model.eval()
            print('')

            # measure FLOPs: need to run this before timing
            flops_pruned = FlopCountAnalysis(
                pruned_model_instance,
                (token_ids_tensor, decoder_input_ids)
            ).total()
            total_flops_pruned += flops_pruned

            # pruned model generation
            model.eval()
            start_time = time.time()
            generated_tokens = model.generate(
                encoder_outputs=BaseModelOutput(
                    last_hidden_state=pruned_model_instance.encoder(token_ids_tensor)
                ),
                decoder_input_ids=decoder_input_ids,
                max_length=128,
                num_beams=4,
                early_stopping=True
            )
            generated_summary = tokenizer.decode(
                generated_tokens[0],
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )
            end_time = time.time()
            total_time += (end_time - start_time)

            flops_ref = FlopCountAnalysis(
                reference_model_instance,
                (token_ids_tensor, decoder_input_ids)
            ).total()
            total_flops_ref += flops_ref

            # reference model generation
            start_time_ref = time.time()
            real_generated_tokens = model.generate(
                input_ids=token_ids_tensor,
                attention_mask=None,
                max_length=128,
                num_beams=4,
                early_stopping=True
            )
            real_generated_summary = tokenizer.decode(
                real_generated_tokens[0],
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )
            end_time_ref = time.time()
            total_time_ref += (end_time_ref - start_time_ref)

            print('')
            print(generated_summary)
            print('----------------')
            print(real_generated_summary)
            print('----------------')
            print(reference_summary)
            print(end_time - start_time, end_time_ref - start_time_ref)
            outputs.append(generated_summary)
            outputs_ref.append(real_generated_summary)
            targets.append(reference_summary)
    average_flops_pruned = total_flops_pruned / len(test_dataset)
    average_flops_ref = total_flops_ref / len(test_dataset)
    average_time = total_time / len(test_dataset)
    average_time_ref = total_time_ref / len(test_dataset)

    rouge_results = rouge.compute(predictions=outputs, references=targets)
    bleu_results = bleu.compute(predictions=outputs, references=targets)

    rouge_results_ref = rouge.compute(predictions=outputs_ref, references=targets)
    bleu_results_ref = bleu.compute(predictions=outputs_ref, references=targets)

    print("Inference Results:")
    print(f"Average forward pass time: {average_time:.4f} seconds")
    print(f"Average forward pass time (reference): {average_time_ref:.4f} seconds")
    print(f"Average FLOPs for pruned model: {average_flops_pruned:.2f}")
    print(f"Average FLOPs for reference model: {average_flops_ref:.2f}")
    print("ROUGE Scores:", rouge_results)
    print("ROUGE Scores (reference):", rouge_results_ref)
    print("BLEU Score:", bleu_results)
    print("BLEU Score (reference):", bleu_results_ref)

    summary_stats = {"average_flops_pruned": average_flops_pruned,
        "average_flops_ref": average_flops_ref,
        "average_time_pruned": average_time,
        "average_time_ref": average_time_ref,
        "average_rouge_pruned": rouge_results,
        "average_rouge_ref": rouge_results_ref,
        "average_bleu_pruned": bleu_results,
        "average_bleu_ref": bleu_results_ref}
    all_results[(w, mc)] = summary_stats

RUNNING WITH NON-ANCHOR WEIGHTING 0.25 AND MERGING-TO-CLOSEST False


  0%|          | 0/100 [00:00<?, ?it/s]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



model.model.shared
  1%|          | 1/100 [00:14<24:00, 14.55s/it]


Sweden's Abba visited the Wellington Museum in Belgium. The city of Waterloo is preparing to mark the 200th anniversary of the battle. The battle was fought in 1815, but Napoleon didn't surrender. The French world still refuses to accept Napoleon as their leader. However, they do visit the famous battlefield.
----------------
'My, my! At Waterloo Napoleon did surrender,’ sang Sweden’s fab four in their famous Eurovision Song Contest winner. But Napoleon certainly didn’t surrender at Waterloo. The British – and the English-speaking nations of the world – remain fascinated by the victory and the bloody manner in which it was achieved.
----------------
Contrary to the Abba  song, Napoleon did not surrender at Waterloo .
Templar hospice is being turned into a restaurant-cum-brewery, museum .
May sees the 75th anniversary of 'glorious failure' of Operation Dynamo .
1.2849431037902832 1.1563951969146729
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



...

model.model.shared
 99%|█████████▉| 99/100 [22:06<00:11, 11.77s/it]


The Basij is a militia loyal to Iran's religious leaders. Iran's elite Guard is already training, advising and supporting Iraqi Shia ISIS. Iran and the U.S. share an enemy in ISIS. But Iran doesn't trust the U., and wants cooperation with it.
----------------
Iran's elite Revolutionary Guard's Quds Force, led by General Qassem Suleimani, is already training, advising and supporting Iraqi Shia militias in their fight against ISIS. Iranians believe air strikes against ISIS are not effective, and feel that the U.S. and its allies are not trying seriously enough.
----------------
Iran's elite Quds Force is training, advising and supporting Iraqi Shia militias in their fight against ISIS.
Iranian officials say they would like better cooperation with the U.S., but say trust between the nations is lacking .
1.1046371459960938 0.9830424785614014
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



model.model.shared
100%|██████████| 100/100 [22:17<00:00, 13.37s/it]


The intruder was arrested after climbing the fence at 10.25pm last night. He was carrying a suspicious package when he jumped the rail. It is the latest in a string of security breaches to hit the White House. Last week an illicitopter was able to land on the lawn of the residence.
----------------
Intruder was arrested after climbing into the grounds of the President's home at 10.25pm. The package was examined and 'deemed to be harmless', according to CNN. It is not known what it contained or why the fence was climbed. The trespasser entered the south side of grounds, which face the National Mall. Last September, a knife-wielding man managed to scale the fence on the north side.
----------------
Trespasser managed to get into grounds of presidential residence .
Secret Service in Washington DC claim they made an 'immediate' arrest .
Intruder's parcel was examined but deemed not to be a risk .
Breach comes a few days after a gyrocopter landed on Capitol's lawn .
1.1223671436309814 1.22

Inference Results:
Average forward pass time: 1.2783 seconds
Average forward pass time (reference): 1.3280 seconds
Average FLOPs for pruned model: 93452045711.36
Average FLOPs for reference model: 134466428712.96
ROUGE Scores: {'rouge1': 0.40518925944706813, 'rouge2': 0.1613742936739437, 'rougeL': 0.26364352614858977, 'rougeLsum': 0.3297466973706315}
ROUGE Scores (reference): {'rouge1': 0.43758022334035174, 'rouge2': 0.20778126724814194, 'rougeL': 0.3106550016641022, 'rougeLsum': 0.36443058843228393}
BLEU Score: {'bleu': 0.12043345845967945, 'precisions': [0.4014175257731959, 0.15749836280288146, 0.07739680426098536, 0.04299255247122546], 'brevity_penalty': 1.0, 'length_ratio': 1.1050195799216802, 'translation_length': 6208, 'reference_length': 5618}
BLEU Score (reference): {'bleu': 0.15643046803911753, 'precisions': [0.38858945707900616, 0.17969894764886107, 0.11070608883488592, 0.0774599698918845], 'brevity_penalty': 1.0, 'length_ratio': 1.3540405838376646, 'translation_length': 7607

In [ ]:
print(all_results)

{(0.25, False): {'average_flops_pruned': 93452045711.36, 'average_flops_ref': 134466428712.96, 'average_time_pruned': 1.2103658413887024, 'average_time_ref': 1.3276606678962708, 'average_rouge_pruned': {'rouge1': 0.3971313871961831, 'rouge2': 0.14922854756327603, 'rougeL': 0.2640525772208051, 'rougeLsum': 0.3246091623625134}, 'average_rouge_ref': {'rouge1': 0.43758022334035174, 'rouge2': 0.20778126724814194, 'rougeL': 0.3106550016641022, 'rougeLsum': 0.36443058843228393}, 'average_bleu_pruned': {'bleu': 0.10923803013444569, 'precisions': [0.3947325372157697, 0.1448528188923998, 0.06764755623203111, 0.03681403750215035], 'brevity_penalty': 1.0, 'length_ratio': 1.0881096475614098, 'translation_length': 6113, 'reference_length': 5618}, 'average_bleu_ref': {'bleu': 0.15643046803911753, 'precisions': [0.38858945707900616, 0.17969894764886107, 0.11070608883488592, 0.0774599698918845], 'brevity_penalty': 1.0, 'length_ratio': 1.3540405838376646, 'translation_length': 7607, 'reference_length': 